### Simon's Exploratory Data Analysis

#### Import Packages

In [1]:
import pandas as pd
from influxdb import DataFrameClient, InfluxDBClient
import pytz
import config
import seaborn as sns
import datetime

localhost:8086


#### Define DataFrameClient from Pandas

In [2]:
client = DataFrameClient(host = config.DB_HOST, port = config.DB_PORT, database = config.DB_DBNAME)

#### Query Data from InfluxDB

In [4]:
#Set time relative to now for Query (today: 00:00:00)
now = datetime.datetime.today()
past = now - datetime.timedelta(days = config.days_delta)

#Set start and end time
end_time = now.strftime("%Y-%m-%d %H:%M:%S")
start_time = past.strftime("%Y-%m-%d %H:%M:%S")

# NoSQL Query  (to be added: timezone adjusting)
query = "SELECT * FROM \"{}\",\"{}\" WHERE time >= '{}' AND time <= '{}' "\
                        .format(config.stations[0], config.stations[1], start_time, end_time)

df_temp = client.query(query)

# to create pandas df, use only one dicitonary part (mythenquai, tiefenbrunnen)
mythenquai_1y = pd.DataFrame(df_temp['mythenquai'])
mythenquai_1y.head()
tiefenbrunnen_1y = pd.DataFrame(df_temp['tiefenbrunnen'])
tiefenbrunnen_1y.head()

,air_temperature,barometric_pressure_qfe,dew_point,humidity,water_temperature,wind_direction,wind_force_avg_10min,wind_gust_max_10min,wind_speed_avg_10min,windchill
2018-11-29 17:20:00+00:00,6.0,969.4,4.5,90,9.5,294,1.4,1.9,1.4,6.0
2018-11-29 17:30:00+00:00,5.9,969.3,4.4,90,9.5,295,1.0,1.5,1.0,5.9
2018-11-29 17:40:00+00:00,5.8,969.3,4.3,90,9.5,299,0.9,1.7,0.9,5.8
2018-11-29 17:50:00+00:00,5.8,969.3,4.3,90,9.5,297,0.7,1.3,0.7,5.8
2018-11-29 18:00:00+00:00,5.8,969.4,4.4,91,9.5,296,0.6,1.2,0.6,5.8
